In [1]:
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text

In [2]:
df = pd.read_excel("/home/valentin_werner/Team2_BPI/MRI_EU_Dataset.xlsx")

In [3]:
df

,Productnumber,Productname,Active Substances,Dose Forms,Reference Member State (RMS),Concerned Member States (CMS),Type Level,Domain,Outcome,Date Of Outcome,Updated At
0,PT/H/0604/004,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,Human,Positive,2012-06-23 02:00:00.000,2022-03-11 13:08:38.733
1,IE/H/0201/003,Escitalopram BioOrganics 20 mgFilm Coated Tablets,escitalopram oxalate 20 mg,Film-coated tablet,IE - Ireland,NaN,Abridged | Additional Strength/Form | Generic ...,Human,Withdrawn,2010-08-11 02:00:00.000,2019-02-26 11:58:53.593
2,DE/H/3956/001,Risedronat AL 75 mg Filmtabletten,risedronate 75 mg,Film-coated tablet,DE - Germany,IT - Italy,Abridged | Initial Application | Generic Art 1...,Human,Positive,2015-04-02 02:00:00.000,2021-08-27 09:45:06.273
3,CZ/V/0160/001,Ralcam 100 mg/ml + 0.05 mg/ml solution for inj...,butafosfan 100 mg/ml | cyanocobalamin 0.05 mg/ml,Solution for injection,CZ - Czechia,DE - Germany,food | Pharmaceutical | Generic - art 13.2 Dir...,Veterinary,Positive,2019-06-03 02:00:00.000,2021-12-22 15:37:22.877
4,AT/H/0667/002,Atorvastatin Accord 20 mg Filmtabletten,atorvastatin calcium trihydrate 20.68 mg,Film-coated tablet,AT - Austria,CY - Cyprus | DE - Germany | DK - Denmark | EE...,Abridged | Initial Application | Generic Art 1...,Human,Positive,2017-06-26 02:00:00.000,2022-05-31 00:41:26.313
...,...,...,...,...,...,...,...,...,...,...,...
53336,ES/H/0864/001,Bupre - 1 A Pharma 35 Mikrogramm/Stunde transd...,buprenorphine 20 mg,Transdermal patch,ES - Spain,PT - Portugal | XI - United Kingdom (Northern ...,[not specified] | [not specified] | [not speci...,Human,Positive,2016-06-02 17:35:23.983,2022-06-17 12:03:24.613
53337,NL/H/5606/001,"Hidroferol 0,266 mg zachte capsules",calcifediol 0.27 mg,"Capsule, soft",NL - Netherlands,BE - Belgium | BG - Bulgaria | DE - Germany | ...,[not specified] | [not specified] | [not speci...,Human,Positive,2017-02-10 13:03:04.913,2022-06-16 19:25:55.757
53338,DE/H/7550/001,Escitalopram Teva20mg/ml druppels voor oraal g...,escitalopram oxalate 20 mg/ml,"Oral drops, solution",DE - Germany,NaN,[not specified] | [not specified] | [not speci...,Human,Positive,2012-11-13 11:22:40.467,2022-06-20 14:51:26.553
53339,DE/H/7551/001,"Escitalopram Teva 20 mg/ml, druppels voor oraa...",escitalopram oxalate 20 mg/ml,"Oral drops, solution",DE - Germany,NaN,[not specified] | [not specified] | [not speci...,Human,Positive,2012-11-13 11:23:23.013,2022-06-20 14:57:12.727


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53341 entries, 0 to 53340
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Productnumber                  53341 non-null  object        
 1   Productname                    53271 non-null  object        
 2   Active Substances              52950 non-null  object        
 3   Dose Forms                     53046 non-null  object        
 4   Reference Member State (RMS)   53341 non-null  object        
 5   Concerned Member States (CMS)  35118 non-null  object        
 6   Type Level                     53341 non-null  object        
 7   Domain                         53341 non-null  object        
 8   Outcome                        53341 non-null  object        
 9   Date Of Outcome                53341 non-null  datetime64[ns]
 10  Updated At                     53341 non-null  datetime64[ns]
dtypes: datetime64[n

In [5]:
df =  df[(df.Outcome == "Positive") & (df.Domain == "Human")]

In [6]:
df.shape

(43759, 11)

In [7]:
df.head()

,Productnumber,Productname,Active Substances,Dose Forms,Reference Member State (RMS),Concerned Member States (CMS),Type Level,Domain,Outcome,Date Of Outcome,Updated At
0,PT/H/0604/004,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,Human,Positive,2012-06-23 02:00:00.000,2022-03-11 13:08:38.733
2,DE/H/3956/001,Risedronat AL 75 mg Filmtabletten,risedronate 75 mg,Film-coated tablet,DE - Germany,IT - Italy,Abridged | Initial Application | Generic Art 1...,Human,Positive,2015-04-02 02:00:00.000,2021-08-27 09:45:06.273
4,AT/H/0667/002,Atorvastatin Accord 20 mg Filmtabletten,atorvastatin calcium trihydrate 20.68 mg,Film-coated tablet,AT - Austria,CY - Cyprus | DE - Germany | DK - Denmark | EE...,Abridged | Initial Application | Generic Art 1...,Human,Positive,2017-06-26 02:00:00.000,2022-05-31 00:41:26.313
5,SE/H/1822/001,Asmanex Twisthaler,mometasone furoate 200 µg,Inhalation powder,SE - Sweden,AT - Austria | BE - Belgium | DE - Germany | D...,[not specified] | [not specified] | [not speci...,Human,Positive,2008-04-05 10:10:51.510,2022-02-02 15:46:37.663
6,NL/H/4961/002,Abiraterone 500 mg,abiraterone acetate 500 mg,Film-coated tablet,NL - Netherlands,LU - Luxembourg,Abridged | Additional Strength/Form | Generic ...,Human,Positive,2021-03-10 01:00:00.000,2022-06-13 17:06:26.517


In [8]:
df = df.drop(columns = ["Productnumber", "Date Of Outcome", "Updated At", "Domain", "Outcome"])

In [9]:
list(df["Type Level"])
#do we need data about prescription / no prescription? I think we dont have it for most other datasets.

['Abridged | Initial Application | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 'Abridged | Initial Application | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 'Abridged | Initial Application | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 '[not specified] | [not specified] | [not specified] | [not specified] | [not specified]',
 'Abridged | Additional Strength/Form | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 'Abridged | Multiple (Copy) Application | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 'Abridged | Initial Application | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 'Abridged | Multiple (Copy) Application | Generic Art 10.1 and 10.2 Dir 2001/83/EC | Chemical Substance | Prescription Only',
 'Abridged | Initial Application | Generic Art 10.1 an

In [10]:
def isNaN(string):
    return string != string

In [11]:
pattern =  "[\d.]+[\s]?(?:mg \/ ml| mg \/ l| mg|mg| g \/ ml| g \/ l| g|g| µg \/ ml| µg \/ l| µg|µg| mcg \/ ml| mcg \/ l| mcg|mcg| ml|ml| l|l)"
strength = []
for info in df["Active Substances"]:
    if isNaN(info) == False:
        found = re.findall(pattern, info)
        strength.append(found)
    else: strength.append([])

In [12]:
df["strength"] = strength

In [13]:
pattern2 = "[[A-z\s().]*?(?=[\d])"
drug = []
for sub in df["Active Substances"]:
    found = re.findall(pattern2, str(sub))
    found_stripped = [text.rstrip().lstrip() for text in found]
    found_new = [found for found in found_stripped if len(found) > 1]
    drug.append(found_new)

/tmp/ipykernel_411152/317314378.py:4: FutureWarning: Possible nested set at position 1
  found = re.findall(pattern2, str(sub))


In [14]:
df["drug"] = drug

In [15]:
list(df.drug)

[['quetiapine fumarate'],
 ['risedronate'],
 ['atorvastatin calcium trihydrate'],
 ['mometasone furoate'],
 ['abiraterone acetate'],
 ['letrozole'],
 ['quetiapine fumarate'],
 ['letrozole'],
 ['atorvastatin calcium trihydrate'],
 ['tamsulosin hydrochloride'],
 ['risperidone'],
 ['mometasone furoate'],
 ['abiraterone acetate'],
 ['mometasone furoate'],
 ['mometasone furoate'],
 ['mometasone furoate'],
 ['risperidone'],
 ['abiraterone acetate'],
 ['amlodipine besilate', 'atorvastatin lysin salt'],
 ['mometasone furoate'],
 ['letrozole'],
 ['rabeprazole'],
 ['tamsulosin hydrochloride'],
 ['risperidone'],
 ['abiraterone acetate'],
 ['calcium phosphate'],
 ['escitalopram oxalate'],
 ['ferrous gluconate'],
 ['rabeprazole'],
 ['montelukast sodium salt'],
 ['calcium phosphate'],
 ['Cetirizine dihydrochloride'],
 ['abiraterone acetate'],
 ['risperidone'],
 ['NORTRIPTYLINE HYDROCHLORIDE'],
 ['escitalopram oxalate'],
 ['alfuzosin hydrochloride'],
 ['montelukast sodium salt'],
 ['colchicine'],
 ['

In [16]:
df.iloc[63]

Productname                                                     Tamoxifen Rosemont
Active Substances                                          Tamoxifen citrate mg/ml
Dose Forms                                                           Oral solution
Reference Member State (RMS)                                          IE - Ireland
Concerned Member States (CMS)                                                  NaN
Type Level                       [not specified] | [not specified] | [not speci...
strength                                                                        []
drug                                                                            []
Name: 69, dtype: object

In [17]:
df = df.reset_index()

In [18]:
patt_countries = "(?<= - )[\w\s]*"
countries = []
for idx in df.index:
    x = []
    for country in re.findall(patt_countries, str(df["Concerned Member States (CMS)"].iloc[idx])): x.append(country)
    for country in re.findall(patt_countries, str(df["Reference Member State (RMS)"].iloc[idx])): x.append(country)

    found_stripped = [text.rstrip().lstrip() for text in x]
    countries.append(found_stripped)

In [19]:
df["country"] = countries

In [20]:
mapper = {"tablet" : "Oral",
        "tablets" : "Oral",
        "capsule" : "Oral",
        "capsules" : "Oral",
        "bottle" : "Oral",
        "kit" : "Oral",
        "oral" : "Oral",
        "syrup" : "Oral",
        "chewing gum" : "Oral",
        "powder" : "Oral",
        "mdi" : "Inhalation",
        "inhalation" : "Inhalation",
        "respules" : "Inhalation",
        "injection" : "Injection",
        "injections" : "Injection",
        "vial" : "Injection",
        "injection infusion" : "Injection",
        "infusion" : "Injection",
        "cartridge" : "Injection",
        "ampoule" : "Injection",
        "solution" : "Injection",
        "syringe" : "Injection",
        "tube" : "Topical",
        "gel" : "Topical",
        "cream" : "Topical",
        "patch" : "Topical",
        "plaster" : "Topical"}
keywords = [
                "tablet", "tablets", "capsule", "capsules", "bottle", "kit", "oral", "syrup", "chewing gum", "powder", #Oral
                "mdi", "respules", "inhalation", #Inhalation
                "injection","injections", "solution", "infusion", "vial", "injection infusion", "cartridge", "syringe", "ampoule", "syringe", #Injection
                "tube", "gel", "cream", "patch", "plaster" #Topical
                ]

In [21]:
mapped = []
for index in df.index:
    row = df["Dose Forms"].iloc[index]
    split = str(row).split()
    z = 0
    for word in split:
        if z == 0:
            word = strip_punctuation(word)
            word = word.lower()
            word = word.rstrip()
            if word in keywords:
                mapped.append(mapper[word.lower()])
                z = 1
    if z == 0:
        mapped.append("nan")

In [22]:
df["route"] = mapped

In [23]:
df["Dose Forms"].iloc[997]

'Medicated plaster'

In [24]:
df

,index,Productname,Active Substances,Dose Forms,Reference Member State (RMS),Concerned Member States (CMS),Type Level,strength,drug,country,route
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],"[Germany, Malta, United Kingdom, Portugal]",Oral
1,2,Risedronat AL 75 mg Filmtabletten,risedronate 75 mg,Film-coated tablet,DE - Germany,IT - Italy,Abridged | Initial Application | Generic Art 1...,[75 mg],[risedronate],"[Italy, Germany]",Oral
2,4,Atorvastatin Accord 20 mg Filmtabletten,atorvastatin calcium trihydrate 20.68 mg,Film-coated tablet,AT - Austria,CY - Cyprus | DE - Germany | DK - Denmark | EE...,Abridged | Initial Application | Generic Art 1...,[20.68 mg],[atorvastatin calcium trihydrate],"[Cyprus, Germany, Denmark, Estonia, Finland, I...",Oral
3,5,Asmanex Twisthaler,mometasone furoate 200 µg,Inhalation powder,SE - Sweden,AT - Austria | BE - Belgium | DE - Germany | D...,[not specified] | [not specified] | [not speci...,[200 µg],[mometasone furoate],"[Austria, Belgium, Germany, Denmark, Spain, Fi...",Inhalation
4,6,Abiraterone 500 mg,abiraterone acetate 500 mg,Film-coated tablet,NL - Netherlands,LU - Luxembourg,Abridged | Additional Strength/Form | Generic ...,[500 mg],[abiraterone acetate],"[Luxembourg, Netherlands]",Oral
...,...,...,...,...,...,...,...,...,...,...,...
43754,53336,Bupre - 1 A Pharma 35 Mikrogramm/Stunde transd...,buprenorphine 20 mg,Transdermal patch,ES - Spain,PT - Portugal | XI - United Kingdom (Northern ...,[not specified] | [not specified] | [not speci...,[20 mg],[buprenorphine],"[Portugal, United Kingdom, Spain]",Topical
43755,53337,"Hidroferol 0,266 mg zachte capsules",calcifediol 0.27 mg,"Capsule, soft",NL - Netherlands,BE - Belgium | BG - Bulgaria | DE - Germany | ...,[not specified] | [not specified] | [not speci...,[0.27 mg],[calcifediol],"[Belgium, Bulgaria, Germany, Estonia, Spain, F...",Oral
43756,53338,Escitalopram Teva20mg/ml druppels voor oraal g...,escitalopram oxalate 20 mg/ml,"Oral drops, solution",DE - Germany,NaN,[not specified] | [not specified] | [not speci...,[20 mg],[escitalopram oxalate],[Germany],Oral
43757,53339,"Escitalopram Teva 20 mg/ml, druppels voor oraa...",escitalopram oxalate 20 mg/ml,"Oral drops, solution",DE - Germany,NaN,[not specified] | [not specified] | [not speci...,[20 mg],[escitalopram oxalate],[Germany],Oral


In [25]:
df_exploded_c = df.explode("country")

In [26]:
df_exploded_c.shape

(171241, 11)

In [27]:
df_exploded_c

,index,Productname,Active Substances,Dose Forms,Reference Member State (RMS),Concerned Member States (CMS),Type Level,strength,drug,country,route
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],Germany,Oral
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],Malta,Oral
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],United Kingdom,Oral
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],Portugal,Oral
1,2,Risedronat AL 75 mg Filmtabletten,risedronate 75 mg,Film-coated tablet,DE - Germany,IT - Italy,Abridged | Initial Application | Generic Art 1...,[75 mg],[risedronate],Italy,Oral
...,...,...,...,...,...,...,...,...,...,...,...
43757,53339,"Escitalopram Teva 20 mg/ml, druppels voor oraa...",escitalopram oxalate 20 mg/ml,"Oral drops, solution",DE - Germany,NaN,[not specified] | [not specified] | [not speci...,[20 mg],[escitalopram oxalate],Germany,Oral
43758,53340,Paracetamol and Tramadol Teva 650/75 mg film-c...,paracetamol 650 mg | tramadol hydrochloride 75 mg,Film-coated tablet,HR - Croatia,ES - Spain | PT - Portugal | SI - Slovenia,[not specified] | [not specified] | [not speci...,"[650 mg, 75 mg]","[paracetamol, tramadol hydrochloride]",Spain,Oral
43758,53340,Paracetamol and Tramadol Teva 650/75 mg film-c...,paracetamol 650 mg | tramadol hydrochloride 75 mg,Film-coated tablet,HR - Croatia,ES - Spain | PT - Portugal | SI - Slovenia,[not specified] | [not specified] | [not speci...,"[650 mg, 75 mg]","[paracetamol, tramadol hydrochloride]",Portugal,Oral
43758,53340,Paracetamol and Tramadol Teva 650/75 mg film-c...,paracetamol 650 mg | tramadol hydrochloride 75 mg,Film-coated tablet,HR - Croatia,ES - Spain | PT - Portugal | SI - Slovenia,[not specified] | [not specified] | [not speci...,"[650 mg, 75 mg]","[paracetamol, tramadol hydrochloride]",Slovenia,Oral


In [28]:
drug_new = []
for drug in df_exploded_c.drug:
    row = ", ".join([drug_part for drug_part in drug])
    drug_new.append(row)

In [29]:
strength_new = []
for strength in df_exploded_c.strength:
    row = ", ".join([strength_part for strength_part in strength])
    strength_new.append(row)

In [30]:
df_exploded_c["strength_str"] = strength_new
df_exploded_c["drug_str"] = drug_new

In [31]:
df_exploded_c

,index,Productname,Active Substances,Dose Forms,Reference Member State (RMS),Concerned Member States (CMS),Type Level,strength,drug,country,route,strength_str,drug_str
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],Germany,Oral,200 mg,quetiapine fumarate
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],Malta,Oral,200 mg,quetiapine fumarate
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],United Kingdom,Oral,200 mg,quetiapine fumarate
0,0,Quetiapina Aurobindo,quetiapine fumarate 200 mg,Film-coated tablet,PT - Portugal,DE - Germany | MT - Malta | XI - United Kingdo...,Abridged | Initial Application | Generic Art 1...,[200 mg],[quetiapine fumarate],Portugal,Oral,200 mg,quetiapine fumarate
1,2,Risedronat AL 75 mg Filmtabletten,risedronate 75 mg,Film-coated tablet,DE - Germany,IT - Italy,Abridged | Initial Application | Generic Art 1...,[75 mg],[risedronate],Italy,Oral,75 mg,risedronate
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43757,53339,"Escitalopram Teva 20 mg/ml, druppels voor oraa...",escitalopram oxalate 20 mg/ml,"Oral drops, solution",DE - Germany,NaN,[not specified] | [not specified] | [not speci...,[20 mg],[escitalopram oxalate],Germany,Oral,20 mg,escitalopram oxalate
43758,53340,Paracetamol and Tramadol Teva 650/75 mg film-c...,paracetamol 650 mg | tramadol hydrochloride 75 mg,Film-coated tablet,HR - Croatia,ES - Spain | PT - Portugal | SI - Slovenia,[not specified] | [not specified] | [not speci...,"[650 mg, 75 mg]","[paracetamol, tramadol hydrochloride]",Spain,Oral,"650 mg, 75 mg","paracetamol, tramadol hydrochloride"
43758,53340,Paracetamol and Tramadol Teva 650/75 mg film-c...,paracetamol 650 mg | tramadol hydrochloride 75 mg,Film-coated tablet,HR - Croatia,ES - Spain | PT - Portugal | SI - Slovenia,[not specified] | [not specified] | [not speci...,"[650 mg, 75 mg]","[paracetamol, tramadol hydrochloride]",Portugal,Oral,"650 mg, 75 mg","paracetamol, tramadol hydrochloride"
43758,53340,Paracetamol and Tramadol Teva 650/75 mg film-c...,paracetamol 650 mg | tramadol hydrochloride 75 mg,Film-coated tablet,HR - Croatia,ES - Spain | PT - Portugal | SI - Slovenia,[not specified] | [not specified] | [not speci...,"[650 mg, 75 mg]","[paracetamol, tramadol hydrochloride]",Slovenia,Oral,"650 mg, 75 mg","paracetamol, tramadol hydrochloride"


In [32]:
df_exploded_c.drop(columns = ["index", "drug", "strength"], inplace = True)

In [33]:
df_exploded_c = df_exploded_c.drop_duplicates()

In [34]:
drug2 = [drug.split(", ") for drug in df_exploded_c.drug_str]
df_exploded_c["drug"] = drug2

strength2 = [strength.split(", ") for strength in df_exploded_c.strength_str]
df_exploded_c["strength"] = strength2

/tmp/ipykernel_411152/1914701796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_c["drug"] = drug2
/tmp/ipykernel_411152/1914701796.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_c["strength"] = strength2


In [35]:
df_exploded_c = df_exploded_c.drop(columns = ['strength_str', 'drug_str', 'Active Substances', 'Dose Forms', 'Reference Member State (RMS)', 'Concerned Member States (CMS)', 'Type Level'])

In [36]:
df_exploded_c = df_exploded_c.reset_index().set_index(keys=["index",'country','Productname','route']).apply(lambda x: x.apply(pd.Series).stack())

In [37]:
df_exploded_c

drug  \
index country        Productname                                        route                             
0     Germany        Quetiapina Aurobindo                               Oral  0     quetiapine fumarate   
      Malta          Quetiapina Aurobindo                               Oral  0     quetiapine fumarate   
      Portugal       Quetiapina Aurobindo                               Oral  0     quetiapine fumarate   
      United Kingdom Quetiapina Aurobindo                               Oral  0     quetiapine fumarate   
1     Germany        Risedronat AL 75 mg Filmtabletten                  Oral  0             risedronate   
...                                                                                                 ...   
43758 Portugal       Paracetamol and Tramadol Teva 650/75 mg film-co... Oral  1  tramadol hydrochloride   
      Slovenia       Paracetamol and Tramadol Teva 650/75 mg film-co... Oral  0             paracetamol   
                                                                              1  tramadol hydrochloride   
      Spain          Paracetamol and Tramadol Teva 650/75 mg film-co... Oral  0             paracetamol   
                                                                              1  tramadol hydrochloride   

                                                                                strength  
index country        Productname                                        route             
0     Germany        Quetiapina Aurobindo                               Oral  0   200 mg  
      Malta          Quetiapina Aurobindo                               Oral  0   200 mg  
      Portugal       Quetiapina Aurobindo                               Oral  0   200 mg  
      United Kingdom Quetiapina Aurobindo                               Oral  0   200 mg  
1     Germany        Risedronat AL 75 mg Filmtabletten                  Oral  0    75 mg  
...                                                                                  ...  
43758 Portugal       Paracetamol and Tramadol Teva 650/75 mg film-co... Oral  1    75 mg  
      Slovenia       Paracetamol and Tramadol Teva 650/75 mg film-co... Oral  0   650 mg  
                                                                              1    75 mg  
      Spain          Paracetamol and Tramadol Teva 650/75 mg film-co... Oral  0   650 mg  
                                                                              1    75 mg  

[219911 rows x 2 columns]

In [38]:
final = df_exploded_c.reset_index().drop(columns = "index")

In [39]:
final

,country,Productname,route,level_4,drug,strength
0,Germany,Quetiapina Aurobindo,Oral,0,quetiapine fumarate,200 mg
1,Malta,Quetiapina Aurobindo,Oral,0,quetiapine fumarate,200 mg
2,Portugal,Quetiapina Aurobindo,Oral,0,quetiapine fumarate,200 mg
3,United Kingdom,Quetiapina Aurobindo,Oral,0,quetiapine fumarate,200 mg
4,Germany,Risedronat AL 75 mg Filmtabletten,Oral,0,risedronate,75 mg
...,...,...,...,...,...,...
219906,Portugal,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,1,tramadol hydrochloride,75 mg
219907,Slovenia,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,0,paracetamol,650 mg
219908,Slovenia,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,1,tramadol hydrochloride,75 mg
219909,Spain,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,0,paracetamol,650 mg


In [42]:
final = final.drop(columns = ["level_4"])

In [43]:
final

,country,Productname,route,drug,strength
0,Germany,Quetiapina Aurobindo,Oral,quetiapine fumarate,200 mg
1,Malta,Quetiapina Aurobindo,Oral,quetiapine fumarate,200 mg
2,Portugal,Quetiapina Aurobindo,Oral,quetiapine fumarate,200 mg
3,United Kingdom,Quetiapina Aurobindo,Oral,quetiapine fumarate,200 mg
4,Germany,Risedronat AL 75 mg Filmtabletten,Oral,risedronate,75 mg
...,...,...,...,...,...
219906,Portugal,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,tramadol hydrochloride,75 mg
219907,Slovenia,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,paracetamol,650 mg
219908,Slovenia,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,tramadol hydrochloride,75 mg
219909,Spain,Paracetamol and Tramadol Teva 650/75 mg film-c...,Oral,paracetamol,650 mg


In [44]:
final.to_csv("/home/valentin_werner/Team2_BPI/EU_Dataset.csv")